# Day 1: Inverse Captcha

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

- `1122` produces a sum of `3` (`1` + `2`) because the first digit (`1`) matches the second digit and the third digit (`2`) matches the fourth digit.
- `1111` produces `4` because each digit (all `1`) matches the next.
- `1234` produces `0` because no digit matches the next.
- `91212129` produces `9` because the only digit that matches the next one is the last digit, `9`.

What is the solution to your captcha?

In [37]:
def checksum_next(digits):
    return sum([int(digit) for (digit, next_digit) 
                in zip(digits, digits[1:]+digits[0]) 
                if digit == next_digit])

In [38]:
[checksum_next(digits) for digits in ["1122", "1111", "1234", "91212129"]]

[3, 4, 0, 9]

## Part Two

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit `10/2 = 5` steps forward matches it. Fortunately, your list has an even number of elements.

For example:

- `1212` produces `6`: the list contains `4` items, and all four digits match the digit `2` items ahead.
- `1221` produces `0`, because every comparison is between a `1` and a `2`.
- `123425` produces `4`, because both `2`s match each other, but no other digit has a match.
- `123123` produces `12`.
- `12131415` produces `4`.

In [24]:
def checksum_rotated(digits):
    offset = len(digits) // 2
    rotated_digits = digits[offset:] + digits[:offset]
    return sum([int(digit) for (digit, matching_digit) 
         in zip(digits, rotated_digits) 
         if digit == matching_digit])

In [25]:
[checksum_rotated(digits) for digits in ["1212", "1221", "123425", "123123", "12131415"]]


[6, 0, 4, 12, 4]